In [2]:
library(data.table)
library(IRanges)

In [3]:
library(parallel)
options(mc.cores = detectCores())

In [4]:
real <- c('A*02:01', 'A*03:01', 'B*13:02', 'B*51:01', 'C*01:02', 'C*06:02', 'DRB1*07:01', 'DRB1*11:01', 'DQB1*02:AB', 'DQB1*03:01', 'DPB1*02:01', 'DPB1*02:01')

In [48]:
dt <- fread('187521910.m8')

Read 5846112 rows and 12 (of 12) columns from 0.421 GB file in 00:00:07


In [49]:
setnames(dt, c('q', 't', 'iden', 'len', 'mis', 'gap', 'qf', 'qt', 'tf', 'tt', 'e', 'score'))
#dt <- dt[iden == 100]
tt <- dt[, gene := sub('\\*.+', '', t)]
setkey(dt, q)

In [50]:
nexpr <- dt[grepl('\\D-', t)]
dt <- dt[grepl('\\d-', t)]
tt <- dt[, c('LEN', 'GOOD', 'GOOD.N', 'BAD.N', 'N') := .(
    max(len), 
    paste(unique(gene[len == max(len)]), collapse = ','),
    sum(len == max(len) & iden == 100),
    sum(len < max(len) | iden < 100),
    .N)
, by = q]

In [195]:
good <- dt[len == LEN & iden == 100]
tt <- good[, pure := gene == GOOD]
dt <- dt[q %in% good$q]
bad <- dt[len < LEN | iden < 100]

In [165]:
ir.score <- function(tf, tt){
    ir <- reduce(IRanges(tf, tt), min.gapwidth = 0)
    return(list(
        nreads = length(tf),
        cov = sum(width(ir)),
        nseg = length(ir),
        nbreaks = sum(width(gaps(ir - 1)) == 2)
    ))
}

In [166]:
ir1 <- good[, ir.score(tf, tt), keyby = t]
ir2 <- good[pure==T, ir.score(tf, tt), keyby = t]
setnames(ir2, c('t', 'nreads2', 'cov2', 'nseg2', 'nbreaks2'))
score <- ir2[ir1][order(-cov)]

In [237]:
get.bad <- function(g){
    bad.score <- bad[gene == g & grepl(sprintf('^%s', g), GOOD), ir.score(tf, tt), keyby = t]
    setnames(bad.score, c('t', 'bad.nreads', 'bad.cov', 'bad.nseg', 'bad.nbreaks'))
    bad.score <- bad.score[score[grepl(sprintf("^%s", g), t)]][order(-cov)]
    return(bad.score)
}
bad.reads <- function(bad, hla){
    dt[q %in% bad[t == hla, q]]
}

In [219]:
head(get.bad('A')[order(bad.nreads/10-cov)])

,t,bad.nreads,bad.cov,bad.nseg,bad.nbreaks,nreads2,cov2,nseg2,nbreaks2,nreads,cov,nseg,nbreaks
1,A*03:01-365,50,286,3,0,34,153,1,0,147,296,3,0
2,A*03:89-365,50,279,4,0,30,144,1,0,148,296,3,0
3,A*03:26-365,51,286,3,0,34,153,1,0,146,294,3,0
4,A*02:01-365,91,254,5,0,54,247,3,0,107,297,2,0
5,A*02:24-365,104,281,5,0,44,247,3,0,97,297,2,0
6,A*02:249-365,105,261,4,0,45,223,3,0,93,296,3,0


In [210]:
head(get.bad('B')[order(bad.nreads/10-cov)])

,t,bad.nreads,bad.cov,bad.nseg,bad.nbreaks,nreads2,cov2,nseg2,nbreaks2,nreads,cov,nseg,nbreaks
1,B*51:01-362,54,237,2,0,17,151,3,0,80,297,2,0
2,B*51:193-362,54,237,2,0,17,151,3,0,80,297,2,0
3,B*51:96-337,53,227,2,0,16,129,2,0,79,295,2,0
4,B*51:187-362,55,237,2,0,17,151,3,0,78,294,3,0
5,B*51:04-362,47,237,2,0,8,99,3,0,71,291,3,0
6,B*51:192-362,66,237,2,0,9,119,3,0,65,292,3,0


In [211]:
head(get.bad('C')[order(bad.nreads/10-cov)])

,t,bad.nreads,bad.cov,bad.nseg,bad.nbreaks,nreads2,cov2,nseg2,nbreaks2,nreads,cov,nseg,nbreaks
1,C*06:107-349,56,245,4,0,4,29,1,0,91,263,4,1
2,C*06:73-273,46,245,4,0,4,29,1,0,105,262,4,1
3,C*06:55-273,39,244,4,0,4,29,1,0,98,261,4,1
4,C*06:02-366,46,245,4,0,4,29,1,0,98,261,4,1
5,C*06:108-273,46,245,4,0,4,29,1,0,98,261,4,1
6,C*06:110-298,46,245,4,0,4,29,1,0,98,261,4,1


In [212]:
head(get.bad('DQB1')[order(bad.nreads/10-cov)])

,t,bad.nreads,bad.cov,bad.nseg,bad.nbreaks,nreads2,cov2,nseg2,nbreaks2,nreads,cov,nseg,nbreaks
1,DQB1*03:01-261,19,131,2,0,52,217,3,1,54,240,4,2
2,DQB1*03:116-261,27,131,2,0,44,212,4,1,46,235,5,2
3,DQB1*03:94-261,26,131,2,0,41,199,3,1,43,222,4,1
4,DQB1*02:02-261,31,131,2,0,34,199,3,1,36,222,4,2
5,DQB1*02:12-261,31,131,2,0,34,199,3,1,36,222,4,2
6,DQB1*02:01-261,32,131,2,0,33,199,4,2,35,222,5,3


In [213]:
head(get.bad('DRB1')[order(bad.nreads/10-cov)])

,t,bad.nreads,bad.cov,bad.nseg,bad.nbreaks,nreads2,cov2,nseg2,nbreaks2,nreads,cov,nseg,nbreaks
1,DRB1*07:01-266,57,200,4,1,3,45,1,0,65,253,4,2
2,DRB1*11:01-266,79,180,3,0,6,81,2,0,69,251,4,1
3,DRB1*14:141-266,79,151,3,1,3,25,1,0,53,251,4,1
4,DRB1*11:04-266,81,216,3,1,6,81,2,0,67,242,4,0
5,DRB1*11:15-266,94,208,3,0,3,25,1,0,54,243,5,1
6,DRB1*11:29-266,94,208,3,0,3,25,1,0,54,243,5,1


In [214]:
head(get.bad('DPB1')[order(bad.nreads/10-cov)])

,t,bad.nreads,bad.cov,bad.nseg,bad.nbreaks,nreads2,cov2,nseg2,nbreaks2,nreads,cov,nseg,nbreaks
1,DPB1*02:01-258,1,36,1,0,160,215,3,2,187,215,3,2
2,DPB1*105:01-258,35,77,1,0,129,213,4,2,153,214,4,2
3,DPB1*02:02-258,40,88,1,0,125,208,4,2,148,208,4,2
4,DPB1*04:02-258,73,133,2,0,113,204,4,2,115,205,4,2
5,DPB1*463:01-258,118,171,2,1,69,190,4,1,70,191,4,1
6,DPB1*04:01-258,52,88,1,0,113,179,4,2,136,179,4,2


In [215]:
head(good)

,q,t,iden,len,mis,gap,qf,qt,tf,tt,e,score,gene,LEN,GOOD,GOOD.N,BAD.N,N,pure
1,H5JCKCCXX:1:12:3432746:0/2,A*01:01-365,100,42,0,0,1,126,250,291,7.6e-21,93.2,A,42,"A,B,C,E,F,G",2370,129,2499,FALSE
2,H5JCKCCXX:1:12:3432746:0/2,A*01:02-365,100,42,0,0,1,126,250,291,7.6e-21,93.2,A,42,"A,B,C,E,F,G",2370,129,2499,FALSE
3,H5JCKCCXX:1:12:3432746:0/2,A*01:03-365,100,42,0,0,1,126,250,291,7.6e-21,93.2,A,42,"A,B,C,E,F,G",2370,129,2499,FALSE
4,H5JCKCCXX:1:12:3432746:0/2,A*01:09-365,100,42,0,0,1,126,250,291,7.6e-21,93.2,A,42,"A,B,C,E,F,G",2370,129,2499,FALSE
5,H5JCKCCXX:1:12:3432746:0/2,A*01:103-273,100,42,0,0,1,126,225,266,7.6e-21,93.2,A,42,"A,B,C,E,F,G",2370,129,2499,FALSE
6,H5JCKCCXX:1:12:3432746:0/2,A*01:104-273,100,42,0,0,1,126,225,266,7.6e-21,93.2,A,42,"A,B,C,E,F,G",2370,129,2499,FALSE


In [216]:
head(bad)

,q,t,iden,len,mis,gap,qf,qt,tf,tt,e,score,gene,LEN,GOOD,GOOD.N,BAD.N,N
1,H5JCKCCXX:1:12:3432746:0/2,A*02:01-365,97.6,42,1,0,1,126,250,291,1.7e-20,92,A,42,"A,B,C,E,F,G",2370,129,2499
2,H5JCKCCXX:1:12:3432746:0/2,A*02:02-365,97.6,42,1,0,1,126,250,291,1.7e-20,92,A,42,"A,B,C,E,F,G",2370,129,2499
3,H5JCKCCXX:1:12:3432746:0/2,A*02:03-365,97.6,42,1,0,1,126,250,291,1.7e-20,92,A,42,"A,B,C,E,F,G",2370,129,2499
4,H5JCKCCXX:1:12:3432746:0/2,A*02:04-365,97.6,42,1,0,1,126,250,291,1.7e-20,92,A,42,"A,B,C,E,F,G",2370,129,2499
5,H5JCKCCXX:1:12:3432746:0/2,A*02:05-365,97.6,42,1,0,1,126,250,291,1.7e-20,92,A,42,"A,B,C,E,F,G",2370,129,2499
6,H5JCKCCXX:1:12:3432746:0/2,A*02:06-365,97.6,42,1,0,1,126,250,291,1.7e-20,92,A,42,"A,B,C,E,F,G",2370,129,2499


In [217]:
head(get.bad('A')[order(bad.nreads/10-cov)])

,t,bad.nreads,bad.cov,bad.nseg,bad.nbreaks,nreads2,cov2,nseg2,nbreaks2,nreads,cov,nseg,nbreaks
1,A*03:01-365,50,286,3,0,34,153,1,0,147,296,3,0
2,A*03:89-365,50,279,4,0,30,144,1,0,148,296,3,0
3,A*03:26-365,51,286,3,0,34,153,1,0,146,294,3,0
4,A*02:01-365,91,254,5,0,54,247,3,0,107,297,2,0
5,A*02:24-365,104,281,5,0,44,247,3,0,97,297,2,0
6,A*02:249-365,105,261,4,0,45,223,3,0,93,296,3,0


In [261]:
hopeless <- function(to.rescue, rescue.by, BAD = bad){
    explained <- unique(good[t %in% rescue.by, q])
    g <- sub('\\*.+', '', to.rescue)
    #BAD[t %in% to.rescue & grepl(sprintf('^%s', g), GOOD) & !(q %in% explained)]
    BAD[t %in% to.rescue & GOOD == g & !(q %in% explained)]
}
rescue <- function(to.rescue, rescue.by, BAD = bad){
    bad.score <- hopeless(to.rescue, rescue.by, BAD)[, ir.score(tf, tt), keyby = t]
    setnames(bad.score, c('t', 'worst.nreads', 'worst.cov', 'worst.nseg', 'worst.nbreaks'))
    bad.score[, rescue.by := rescue.by]
    return(bad.score)
}
a <- 'A*02:01-365'
b <- 'A*03:01-365'
rescue(b, a)

,t,worst.nreads,worst.cov,worst.nseg,worst.nbreaks,rescue.by
1,A*03:01-365,4,85,2,0,A*02:01-365


In [262]:
system.time(rescue(a, b))

   user  system elapsed 
  0.148   0.000   0.147 

In [263]:
badA <- bad[gene == 'A' & grepl('^A', GOOD)]
system.time(rescue(a, b, badA))

   user  system elapsed 
  0.024   0.000   0.025 

In [264]:
scoreA <- get.bad('A')
scoreA <- scoreA[cov/max(cov) > 0.95]
nrow(scoreA)

[1] 45

In [265]:
rescue(a, b, badA)

,t,worst.nreads,worst.cov,worst.nseg,worst.nbreaks,rescue.by
1,A*02:01-365,5,110,2,0,A*03:01-365


In [266]:
rescue(b, a, badA)

,t,worst.nreads,worst.cov,worst.nseg,worst.nbreaks,rescue.by
1,A*03:01-365,4,85,2,0,A*02:01-365


In [270]:
rescue('A*03:89-365', b, badA)

,t,worst.nreads,worst.cov,worst.nseg,worst.nbreaks,rescue.by
1,A*03:89-365,13,152,3,0,A*03:01-365


In [259]:
good[q == 'H5JCKCCXX:3:6:4089392:0/2']

,q,t,iden,len,mis,gap,qf,qt,tf,tt,e,score,gene,LEN,GOOD,GOOD.N,BAD.N,N,pure
1,H5JCKCCXX:3:6:4089392:0/2,A*02:137-273,100,49,0,0,149,3,121,169,1e-24,106.3,A,49,A,609,1,610,TRUE
2,H5JCKCCXX:3:6:4089392:0/2,A*02:24-365,100,49,0,0,149,3,146,194,1e-24,106.3,A,49,A,609,1,610,TRUE
3,H5JCKCCXX:3:6:4089392:0/2,A*02:503-181,100,49,0,0,149,3,121,169,1e-24,106.3,A,49,A,609,1,610,TRUE
4,H5JCKCCXX:3:6:4089392:0/2,A*29:51-273,100,49,0,0,149,3,121,169,1e-24,106.3,A,49,A,609,1,610,TRUE
